# Artificial Neural Network

### Importing the libraries

In [38]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [39]:
tf.__version__

'2.15.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [40]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values #column 0,1,2 are irrelevant for churn modelling and removing the last column too
y = dataset.iloc[:, -1].values #dependent variable

In [41]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [42]:
print(y)

[1 0 1 ... 1 1 0]


### Encoding categorical data

Label Encoding the "Gender" column

In [43]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder() #creating an instance le of LabelEncoder() class
X[:, 2] = le.fit_transform(X[:, 2]) #taking all rows of second column from X, i.e. Gender and fit transform it using label encdoder

In [44]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" column

In [45]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough') #one hot encoding on column 1
X = np.array(ct.fit_transform(X))

In [46]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting the dataset into the Training set and Test set

In [47]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature Scaling (Important for any Deep Learning model)

In [48]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train) #applying feature scaling to all the features of both training set and test set
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [49]:
ann = tf.keras.models.Sequential() #creating an object of Sequential class from models module of tensorflow keras

### Adding the input layer and the first hidden layer

In [50]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu')) # 6 neurons in the first hidden layer (Dense = fully connected)

### Adding the second hidden layer

In [51]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu')) # 6 neurons in the second hidden layer

### Adding the output layer

In [52]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))  #only 1 output neuron needed to classify as 0 or 1 so used sigmoid activation function

## Part 3 - Training the ANN

### Compiling the ANN

In [53]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
#compile method used to compile the model. 'adam' optimizer can perform stocashtic gradient descent very well. (NOTE: for binary classification the loss function should be 'binary_crossentropy' and for non-binary classification the loss function should be 'categorical_crossentropy').
#metric contains the list various metrics with which we want to evaluate our ann (Here, we have choosed 'accuracy').

### Training the ANN on the Training set

In [54]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)
# fit method used to train, whatever the ML model is. fit method takes 4 parameters, i.e. independent part of training set, dependent part of the training set, batch_size and epochs.

Epoch 1/100
250/250 [==============================] - 2s 2ms/step - loss: 0.5803 - accuracy: 0.7079
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4722 - accuracy: 0.7977
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4470 - accuracy: 0.8073
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4367 - accuracy: 0.8126
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4309 - accuracy: 0.8135
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4270 - accuracy: 0.8163
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4236 - accuracy: 0.8184
Epoch 8/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4197 - accuracy: 0.8231
Epoch 9/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4157 - accuracy: 0.8271
Epoch 10/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4120 - accura

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank:

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer?

**Solution**

In [55]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

# Calling the predict method. It is very important to remember that any input to the predict method must be a 2D array. 1, 0, 0 is for encoded geography.
# The predict method need to be called on the observations onto which the same scaling was applied as in the training.
# (NOTE: don't use the fit transform here, because the fit transform is used to get the mean and standard deviation of the values in the training set in order to fit the scale to the training set.)
# So, for test set, we only need to call the transform method because if we fit it again then it would lead to some information leakage.
# at last using threshold of 0.5, because the predict returns the actual predicted probability.


1/1 [==============================] - 0s 130ms/step
[[False]]


Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predicting the Test set results

In [56]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 1ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [57]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1517   78]
 [ 202  203]]


0.86